# Latent Dirichlet Allocation

In [1]:
import pandas as pd
from gensim import corpora, models
from ast import literal_eval

import pandas as pd
import numpy as np
import json

In [2]:
data = pd.read_csv('../out/01-preprocessing.tsv', sep='\t')
data = data[['raw_content', 'username', 'mention', 'cleaned_content']]
data['mention'] = data['mention'].apply(literal_eval)
data_by_mention = data.explode('mention').reset_index(drop=True).copy(deep=True)

In [4]:
data_coldplay = []

In [5]:
for username, df in data.groupby(by='username'):
    if username != 'coldplay':
        continue
    data_coldplay.append(df)

In [7]:
for username, df in data_by_mention.groupby(by='mention'):
    if username != 'coldplay':
        continue
    data_coldplay.append(df)

In [14]:
df_coldplay = pd.concat(data_coldplay).reset_index(drop=True)
df_coldplay

,raw_content,username,mention,cleaned_content
0,@rrzzaa__ @IDWantsColdplay I’m so happy to hea...,coldplay,"[rrzzaa__, IDWantsColdplay, thepauldugdale]","['i', 'm', 'so', 'happy', 'to', 'hear', 'you',..."
1,"Huhu ada lagiii, HALOOO @coldplay https://t.co...",bekstreettt,coldplay,"['huhu', 'lagiii', 'halooo']"
2,Salah satu tanda @coldplay ke Indonesia untuk ...,SeleccionINA,coldplay,"['salah', 'tanda', 'indonesia', 'tur', 'dunia'..."
3,Salah satu tanda @coldplay ke Indonesia untuk ...,SeleccionINA,coldplay,"['salah', 'tanda', 'indonesia', 'tur', 'dunia'..."
4,@IDWantsColdplay @coldplay Bismillah menang wa...,finyjr,coldplay,"['bismillah', 'menang', 'war', 'tiket', 'aamiin']"
...,...,...,...,...
1693,@GRhs292374 @IDWantsColdplay @coldplay Hi kaa ...,Silvieoctavian1,coldplay,"['hi', 'kaa', 'buka', 'yah', 'testimony', 'cek..."
1694,@IDWantsColdplay @coldplay Amin 1 tiket😇,everglowstuf,coldplay,"['amin', 'tiket']"
1695,Duetin @coldplay sama @gloriajessicaID di Jaka...,endehendeh,coldplay,"['duetin', 'jakarta', 'coldplayinjakarta']"
1696,@EmirFateh @anwaribrahim @coldplay Zaman Pas P...,mandalorian1605,coldplay,"['zaman', 'pn', 'konsert', 'billie', 'eiliah',..."


In [16]:
content = list(df_coldplay['cleaned_content'].apply(literal_eval))
dictionary = corpora.Dictionary(content)
corpus = [dictionary.doc2bow(doc) for doc in content]
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [17]:
lda_model.show_topics(num_topics=3, num_words=5, formatted=False)

[(0,
  [('ga', 0.013505594),
   ('nonton', 0.011004235),
   ('tiket', 0.0096602375),
   ('jakarta', 0.009237548),
   ('gak', 0.008041168)]),
 (1,
  [('tiket', 0.038210172),
   ('war', 0.022726862),
   ('allah', 0.011174999),
   ('bang', 0.009729713),
   ('bca', 0.0080764545)]),
 (2,
  [('coldplay', 0.018884676),
   ('konser', 0.018717457),
   ('tiket', 0.013730012),
   ('kak', 0.009837772),
   ('nonton', 0.008964083)])]

## Export TO JSON

In [65]:
data_output = {}

In [66]:
for username, df in data.groupby(by='username'):
    # if len(df) <= 1:
    #     continue
    content = list(df['cleaned_content'].apply(literal_eval))
    mention = list(df['mention'])
    raw_content = list(df['raw_content'])
    
    try:
        dictionary = corpora.Dictionary(content)
        corpus = [dictionary.doc2bow(doc) for doc in content]
        lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary)
    except:
        print(username, dictionary, corpus)
    
    topic = []
    for i, x in lda_model.show_topics(num_topics=3, num_words=5, formatted=False):
        topic.append([{'word': word, 'score': str(round(score, 4))} for word, score in x])
        
    data_output[username.lower()] = {
        'topic': topic,
        'source': [{
            'username': username,
            'content': c
        } for c in zip(raw_content)]
    }

BrandZe1 Dictionary<0 unique tokens: []> [[]]
SORAZKl Dictionary<0 unique tokens: []> [[]]
agungmuhar11194 Dictionary<0 unique tokens: []> [[]]
ambilbulan Dictionary<0 unique tokens: []> [[]]
arthrtitis Dictionary<0 unique tokens: []> [[]]
bedoeL28 Dictionary<0 unique tokens: []> [[]]
brrrmi Dictionary<0 unique tokens: []> [[]]
deyybel Dictionary<0 unique tokens: []> [[]]
farabira Dictionary<0 unique tokens: []> [[]]
freakygurlll_ Dictionary<0 unique tokens: []> [[]]
gervasiusbayu Dictionary<0 unique tokens: []> [[]]
gusashdarma Dictionary<0 unique tokens: []> [[]]
harybro Dictionary<0 unique tokens: []> [[]]
ismettaufik Dictionary<0 unique tokens: []> [[]]
mo0nphas3 Dictionary<0 unique tokens: []> [[]]
moerss41 Dictionary<0 unique tokens: []> [[]]
moneyheist_____ Dictionary<0 unique tokens: []> [[]]
rrrizzk Dictionary<0 unique tokens: []> [[]]
tabroni177 Dictionary<0 unique tokens: []> [[]]
thattangerine Dictionary<0 unique tokens: []> [[]]


In [67]:
for username, df in data_by_mention.groupby(by='mention'):
    # if len(df) <= 1:
    #     continue
    content = list(df['cleaned_content'].apply(literal_eval))
    mentioned = list(df['username'])
    raw_content = list(df['raw_content'])
    
    try:
        dictionary = corpora.Dictionary(content)
        corpus = [dictionary.doc2bow(doc) for doc in content]
        lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary)
    except:
        print(username, dictionary, corpus)
    
    topic = []
    for i, x in lda_model.show_topics(num_topics=3, num_words=5, formatted=False):
        topic.append([{'word': word, 'score': str(round(score, 4))} for word, score in x])
        
    data_output[username.lower()] = {
        'topic': topic,
        'source': [{
            'username': m,
            'content': c
        } for m, c in zip(mentioned,raw_content)]
    }

TYOrivera Dictionary<0 unique tokens: []> [[]]
anggi_anggoy Dictionary<0 unique tokens: []> [[]]
nadiaaaps_ Dictionary<0 unique tokens: []> [[]]


In [68]:
output = {
    'data': [{'username': k, 'topic': v['topic'], 'source': v['source']} for k, v in data_output.items()]
}

In [69]:
with open('../out/02-lda-v2.json', 'w') as f:
    json.dump(output, f, indent=4)